# Reproducibility
***
You can follow this Jupyter Notebook in order to reproduce the figure 3 of our paper about reproducibility in bioinformatics ([Kim et al. BioRxiv 2017](http://www.biorxiv.org/content/early/2017/06/20/143503)).

Our StratiPy project is based on NBS (Network Based Stratification) method ([Hofree et al. Nat. Meth. 2013](http://www.nature.com/nmeth/journal/v10/n11/full/nmeth.2651.html)).

# 1 - Parameters
---
There are several tuning parameters of NBS. In order to reproduce same result, you only use same values outlined in the original NBS study except two ajustable parameters.

1. **Graph regulator factor (lambda)** is the most influential parameter in this case study. It was thought that this factor had to be a constant value of 1 until we found that its value was changing and converged to 1800. Here you run NBS with lambd = 1 and 1800 in order to compare them.
2. **Permutation number of bootstrap** utilized in the original study is 1000. But we recommand to start with 100 permutations since this step is highly time consuming (optimization ongoing ). In this notebook, you will notice no significant difference between 100 and 1000 permutations of original results (MATLAB). However you can always lunch with 1000 permutations if you want.

Details about each parameter are explained in docstring of [reproducibility.py](reproducibility.py).

In [1]:
# Defalut settings
data_folder = '../data/' 
patient_data = 'TCGA_UCEC' # Uterine endometrial carcinoma (uterine cancer) with 248 patients' somatic mutation data
ppi_data = 'STRING' # STRING PPI network database
influence_weight = 'min' # Influence weight of propagation on the network
simplification = True # Simplification after propagation
compute = True
overwrite = False
alpha = 0.7 # Diffusion (propagation) factor
tol = 10e-3 # Convergence threshold during diffusion
ngh_max = 11 # Number of best influencers in PPI network
keep_singletons = False
min_mutation = 10
max_mutation = 200000
qn = 'mean' # Quantile normalization (QN) after diffusion is based on the mean of ranked values
n_components = 3 # Desired number of subgroups (clusters)
run_bootstrap = True
run_consensus = True
tol_nmf = 1e-3 # Convergence threshold of NMF and GNMF algorithm
linkage_method = 'average' # Linkage method of hierarchical clustering

## Code

In [3]:
import sys
import os
import nbs_functions, confusion_matrices
sys.path.append(os.path.dirname(os.path.abspath('.')))
from stratipy import load_data, formatting_data, filtering_diffusion, clustering, hierarchical_clustering
import scipy.sparse as sp
from scipy.io import loadmat, savemat
import numpy as np
import time
import datetime

In [8]:
def nbs_functions(data_folder, patient_data, ppi_data, influence_weight, simplification,
                 compute, overwrite, alpha, tol, ngh_max, keep_singletons, min_mutation,
                 max_mutation, qn, n_components, n_permutations, run_bootstrap, run_consensus,
                 lambd, tol_nmf, linkage_method):
    if (sys.version_info < (3, 2)):
        raise "Must be using Python ≥ 3.2"

    else:
        result_folder = 'reproducibility_data/' + 'result_' + patient_data + '_' + ppi_data + '/'
        print('\n######################## Starting StratiPy ########################')
        print("\nGraph regulator factor (lambda) =", lambd)
        print("Permutation number of bootstrap =", n_permutations)

        print("\n------------ load_data.py ------------ {}"
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        (patient_id, mutation_profile, gene_id_patient,
         gene_symbol_profile) = load_data.load_TCGA_UCEC_patient_data(
             data_folder)

        gene_id_ppi, network = load_data.load_PPI_String(data_folder, ppi_data)

        print("\n------------ formatting_data.py ------------ {}"
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        (network, mutation_profile,
         idx_ppi, idx_mut, idx_ppi_only, idx_mut_only) = (
            formatting_data.classify_gene_index(
                network, mutation_profile, gene_id_ppi, gene_id_patient))

        (ppi_total, mut_total, ppi_filt, mut_filt) = (
            formatting_data.all_genes_in_submatrices(
                network, idx_ppi, idx_mut, idx_ppi_only, idx_mut_only,
                mutation_profile))

        print("\n------------ filtering_diffusion.py ------------ {}"
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        final_influence = (
            filtering_diffusion.calcul_final_influence(
                sp.eye(ppi_filt.shape[0], dtype=np.float32), ppi_filt,
                result_folder, influence_weight, simplification,
                compute, overwrite, alpha, tol))

        ppi_final, mut_final = filtering_diffusion.filter_ppi_patients(
            ppi_total, mut_total, ppi_filt, final_influence, ngh_max,
            keep_singletons, min_mutation, max_mutation)

        mut_type, mut_propag = filtering_diffusion.propagation_profile(
            mut_final, ppi_filt, result_folder, alpha, tol, qn)

        # ------------ clustering.py ------------
        print("\n------------ clustering.py ------------ {}"
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        genes_clustering, patients_clustering = (clustering.bootstrap(
            result_folder, mut_type, mut_propag, ppi_final,
            influence_weight, simplification,
            alpha, tol, keep_singletons, ngh_max, min_mutation, max_mutation,
            n_components, n_permutations,
            run_bootstrap, lambd, tol_nmf))

        distance_genes, distance_patients = clustering.consensus_clustering(
            result_folder, genes_clustering, patients_clustering,
            influence_weight, simplification, mut_type,
            alpha, tol, keep_singletons, ngh_max, min_mutation, max_mutation,
            n_components, n_permutations, run_consensus, lambd, tol_nmf)

        # ------------ hierarchical_clustering.py ------------
        print("\n------------ hierarchical_clustering.py ------------ {}"
              .format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        hierarchical_clustering.distance_patients_from_consensus_file(
            result_folder, distance_patients, ppi_data, mut_type,
            influence_weight, simplification, alpha, tol,  keep_singletons,
            ngh_max, min_mutation, max_mutation, n_components, n_permutations,
            lambd, tol_nmf, linkage_method)

## Let's launch!

### With Graph regulator factor (lambda) = 1

In [5]:
lambd = 1
n_permutations = 100

### ! ! ! Clustering part will take few hours ! ! !

In [9]:
lambd1_perm100 = nbs_functions.all_functions(data_folder, patient_data, ppi_data, influence_weight,
                               simplification, compute, overwrite, alpha, tol, ngh_max,
                               keep_singletons, min_mutation, max_mutation, qn, n_components,
                               n_permutations, run_bootstrap, run_consensus, lambd, tol_nmf,
                               linkage_method)


######################## Starting StratiPy ########################

Graph regulator factor (lambda) = 1
Permutation number of bootstrap = 100


NameError: name 'datetime' is not defined

### With Graph regulator factor (lambda) = 1800

In [15]:
lambd = 1800
n_permutations = 100

### ! ! ! Clustering part will take few hours ! ! !  (yes, once again) 

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
lambd1800_perm100 = nbs_functions.all_functions(data_folder, patient_data, ppi_data, influence_weight,
                                  simplification, compute, overwrite, alpha, tol, ngh_max,
                                  keep_singletons, min_mutation, max_mutation, qn,
                                  n_components, n_permutations, run_bootstrap, run_consensus,
                                  lambd, tol_nmf, linkage_method)


######################## Starting StratiPy ########################

Graph regulator factor (lambda) = 1
Permutation number of bootstrap = 100

------------ load_data.py ------------ 2017-09-22 18:05:34
 ==== TCGA patients' ID  
 ==== TCGA mutation profiles  
 ==== TCGA Entrez gene ID and gene symbols in mutation profiles  
 ==== load_PPI_String and gene_id_ppi
 ==== load_PPI 

------------ formatting_data.py ------------ 2017-09-22 18:05:35
 ==== classify_gene_index  
 ==== all_genes_in_submatrices 
 ==== ABC  
 ==== mut_total  
 ==== filter only genes in PPI  
 ==== all_genes_in_submatrices finish  

------------ filtering_diffusion.py ------------ 2017-09-22 18:05:39
 **** Same parameters file of FINAL INFLUENCE already exists
 Removing 0 patients with less than 10 or more than 200000 mutations
 **** Same parameters file of FINAL INFLUENCE on Mutation Profile already exists

------------ clustering.py ------------ 2017-09-22 18:05:46
 **** Same parameters file of bootstrap already 

### Load data

In [ ]:
# Hofree's result --- 100 and 1000 permutations of bootstrap
hof_100_data = loadmat('100permutations/results_NBS_Hofree_100.mat')
hof_1000_data = loadmat('1000permutations/results_NBS_Hofree_1000.mat')

# StratiPy's result --- 100 and 1000 permutations of bootstrap
stp_100_data = loadmat('100permutations/hierarchical_clustering_Patients_weight=min_simp=True_alpha=0.7_tol=0.001_singletons=False_ngh=11_minMut=10_maxMut=200000_comp=3_permut=100_lambd=1_tolNMF=0.001_method=average.mat')
stp_1000_data = loadmat('1000permutations/hierarchical_clustering_Patients_weight=min_simp=True_alpha=0.7_tol=0.001_singletons=False_ngh=11_minMut=10_maxMut=200000_comp=3_permut=1000_lambd=1_tolNMF=0.001_method=average.mat')

# StratiPy's result with lambda = 1800
stp_100_data_lamb1800 = loadmat('100permutations/lamb1800/hierarchical_clustering_Patients_weight=min_simp=True_alpha=0.7_tol=0.001_singletons=False_ngh=11_minMut=10_maxMut=200000_comp=3_permut=100_lambd=1800_tolNMF=0.001_method=average.mat')

data check

In [ ]:
hof_100_data

In [ ]:
stp_100_data

In [ ]:
for key, value in hof_100_data.items() :
    print (key)

In [ ]:
hof_1000_data['propnmf_options']

In [ ]:
hof_100_data['NBS_cc_label'].shape

In [ ]:
stp_100_data['flat_cluster_number'].shape

In [ ]:
hof_1000_data['NBS_cc_label'].shape

### Get number of subgroups for each patient 
there are 3 subroups

In [ ]:
# Hofree
hof_100_3cluster = hof_100_data['NBS_cc_label'].squeeze().tolist()
hof_1000_3cluster = hof_1000_data['NBS_cc_label'].squeeze().tolist()

# StratiPy
stp_100_3cluster = list(stp_100_data['flat_cluster_number'][0])
stp_1000_3cluster = list(stp_1000_data['flat_cluster_number'][0])
stp_100_lamb1800_3cluster = list(stp_100_data_lamb1800['flat_cluster_number'][0])

In [ ]:
def replace_list_element(l, before, after):
    for i,e in enumerate(l):
        if e==before:
            l[i]=after
    return l

In [ ]:
# clust(Hofree) 3(1) <-> clust(Stratipy) 1(3)
stp_100_3cluster_1 = replace_list_element(stp_100_3cluster, 3, 0) # 3 -> 0
stp_100_3cluster_2 = replace_list_element(stp_100_3cluster_1, 1, 3) # 1 -> 3
stp_100_3cluster_3 = replace_list_element(stp_100_3cluster_2, 0, 1) # 0 -> 1

### Confusion matrices

In [ ]:
from sklearn.metrics import confusion_matrix

#### 100 vs 1000 permutations of Bootstrap

In [ ]:
# Hofree 100 & 1000
confusion_matrix(hof_100_3cluster, hof_1000_3cluster)

In [ ]:
# 100 permutations
confusion_matrix(stp_100_3cluster, stp_1000_3cluster)

=> There are no significant differences between 100 and 1000 permutations of Bootstrap.

#### Hofree vs StratiPy

In [ ]:
# 100 permutations
# StratiPy's lambd = 1
confusion_matrix(hof_100_3cluster, stp_100_3cluster)

In [ ]:
# 1000 permutations
# StratiPy's lambd = 1
confusion_matrix(hof_1000_3cluster, stp_1000_3cluster)

In [ ]:
# 100 permutations
# StratiPy's lambd = 1800
confusion_matrix(hof_100_3cluster, stp_100_lamb1800_3cluster)

### Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
conf_arr = confusion_matrix(hof_100_3cluster, stp_100_3cluster)
conf_arr

In [ ]:
conf_arr = np.array([[54,  7,  0],
                  [ 5,  99, 0],
                  [ 0, 54, 29]])

conf_arr = conf_arr.astype('float') / conf_arr.sum(axis=1)[:, np.newaxis]
conf_arr = np.around(conf_arr, decimals=2)
conf_arr

In [ ]:
norm_conf = []
for i in conf_arr:
    a = 0
    tmp_arr = []
    a = sum(i, 0)
    for j in i:
        tmp_arr.append(float(j)/float(a))
    norm_conf.append(tmp_arr)

fig = plt.figure()
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = ax.imshow(np.array(norm_conf), cmap=plt.cm.viridis, interpolation='nearest') #plt.cm.viridis

width, height = conf_arr.shape

for x in range(width):
    for y in range(height):
        ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')

levels = np.linspace(0, 1, 11, endpoint=True)
cb = fig.colorbar(res, ticks=levels, norm=colors.NoNorm)
cb.set_clim(vmin=0, vmax=0.98)
alphabet = '123'
plt.xticks(range(width), alphabet[:width])
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.yticks(range(height), alphabet[:height])

plt.xlabel('Subgroups')
plt.title("Reported tuning parameter value \n\n", fontsize=14)

# plt.savefig('confusion_matrix_lambd1.pdf', bbox_inches='tight')
# plt.savefig('confusion_matrix_lambd1.svg', bbox_inches='tight')

plt.show()

In [ ]:
conf_arr = confusion_matrix(hof_100_3cluster, stp_100_lamb1800_3cluster)
conf_arr = conf_arr.astype('float') / conf_arr.sum(axis=1)[:, np.newaxis]
conf_arr = np.around(conf_arr, decimals=2)
conf_arr

In [ ]:
norm_conf = []
for i in conf_arr:
    a = 0
    tmp_arr = []
    a = sum(i, 0)
    for j in i:
        tmp_arr.append(float(j)/float(a))
    norm_conf.append(tmp_arr)

fig = plt.figure()
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = ax.imshow(np.array(norm_conf), cmap=plt.cm.viridis, interpolation='nearest') #plt.cm.viridis

width, height = conf_arr.shape

for x in range(width):
    for y in range(height):
        ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')

cb = fig.colorbar(res, ticks=levels)
# cb.set_clim(vmin=0, vmax=0.98)

alphabet = '123'
plt.xticks(range(width), alphabet[:width])
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.yticks(range(height), alphabet[:height])

plt.xlabel('Subgroups')
plt.title("Tuning parameter value in code \n\n", fontsize=14)

# plt.savefig('confusion_matrix_lambd1800.pdf', bbox_inches='tight')
# plt.savefig('confusion_matrix_lambd1800.svg', bbox_inches='tight')

plt.show()